# PapyrusVision Hieroglyph Detection - Model Evaluation

This notebook provides evaluation of the trained hieroglyph detection model, analyzing its performance across different metrics and providing insights into detection capabilities.

## What you'll see:

1. **Model Loading & Verification**
   - Load the trained Detectron2 model
   - Verify model files and configuration
   - Check model metadata and training parameters

2. **Performance Metrics**
   - Overall detection accuracy (mAP@0.5, mAP@0.5:0.95)
   - Per-class performance analysis
   - Confidence score distributions
   - Detection statistics by Gardiner codes

3. **Visual Analysis**
   - Sample predictions with bounding boxes
   - Confusion matrix for common classes
   - Detection confidence visualizations
   - Error analysis (false positives/negatives)

4. **Class-Specific Evaluation**
   - Performance breakdown by hieroglyph categories (A, B, D, F, G, etc.)
   - Identification of challenging classes
   - Analysis of class imbalance impact

5. **Model Robustness Testing**
   - Performance across different confidence thresholds
   - Detection quality on various image regions
   - Analysis of small vs. large hieroglyphs

6. **Comparison & Insights**
   - Training vs. validation performance
   - Model strengths and limitations
   - Recommendations for improvement

## Setup

In [ ]:
# Install dependencies
!pip install -U torch torchvision cython
!pip install -U 'git+https://github.com/facebookresearch/fvcore.git' 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
import torch, torchvision
torch.__version__

In [ ]:
!git clone https://github.com/facebookresearch/detectron2 detectron2_repo
!pip install -e detectron2_repo

In [ ]:
# Import libraries
import os
import sys
import json
import numpy as np
import matplotlib.pyplot as plt
import glob

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Set up paths
PROJECT_ROOT = '/content/drive/My Drive/PapyrusNU_Detectron'
DATA_DIR = f'{PROJECT_ROOT}/data'
MODELS_DIR = f'{PROJECT_ROOT}/models'

print(f'Project root: {PROJECT_ROOT}')
print(f'Data directory: {DATA_DIR}')
print(f'Models directory: {MODELS_DIR}')

## Model Loading

In [ ]:
# Find the most recent trained model
model_dirs = glob.glob(f'{MODELS_DIR}/hieroglyph_model_*')
if not model_dirs:
    print(' No trained models found!')
    print('Please run the training notebook (03_model_training.ipynb) first.')
    raise FileNotFoundError('No trained models found')

# Use the most recent model
latest_model_dir = sorted(model_dirs)[-1]
print(f'Using model from: {latest_model_dir}')

# Check if model_info.json exists
model_info_file = f'{latest_model_dir}/model_info.json'
if os.path.exists(model_info_file):
    with open(model_info_file, 'r') as f:
        model_info = json.load(f)
    print(f' Model info loaded successfully!')
    print(f'  Number of classes: {model_info["num_classes"]}')
    print(f'  Detection threshold: {model_info["detection_threshold"]}')
    print(f'  Timestamp: {model_info["timestamp"]}')
else:
    print(f' Model info file not found: {model_info_file}')
    print('The model_info.json file was just created. Please try again.')

## Verify Model Files

In [ ]:
# Check what files exist in the model directory
print(f'Contents of model directory: {latest_model_dir}')
if os.path.exists(latest_model_dir):
    for file in os.listdir(latest_model_dir):
        file_path = os.path.join(latest_model_dir, file)
        if os.path.isfile(file_path):
            size_mb = os.path.getsize(file_path) / (1024 * 1024)
            print(f'  {file}: {size_mb:.1f} MB')
        else:
            print(f'  {file}/ (directory)')
else:
    print(f' Model directory does not exist: {latest_model_dir}')

## Test Data Verification

In [ ]:
# Check test data files
test_annotations_file = f'{DATA_DIR}/annotations/test_annotations.json'
if os.path.exists(test_annotations_file):
    with open(test_annotations_file, 'r') as f:
        test_data = json.load(f)
    print(f' Test data loaded successfully!')
    print(f'  Images: {len(test_data["images"])}')
    print(f'  Annotations: {len(test_data["annotations"])}')
    print(f'  Categories: {len(test_data["categories"])}')

    # Check if images exist
    sample_image = test_data['images'][0]['file_name']
    image_path = f'{DATA_DIR}/images/{sample_image}'
    if os.path.exists(image_path):
        print(f' Sample image exists: {sample_image}')
    else:
        print(f' Sample image not found: {image_path}')
else:
    print(f' Test annotations file not found: {test_annotations_file}')